In [3]:
# import libraries needed for the project

#data handling tools
import pandas as pd
import numpy as np

#data visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

#machine learning tools
# TensorFlow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TextVectorization, Embedding, Input, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import GlobalAvgPool1D, LSTM, GRU, Bidirectional, Conv1D
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# other libraries
import random

In [4]:
# import the training and testing data
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')

In [5]:
# read the head of the dataset to become familiar with the data
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


since we're not working with a sequential data, we might want to shuffle to prevent the models from learning in some order

In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# looking at the test set
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# get 5 random tweets and their classes
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---')

Target: 0 (not real disaster)
Text:
Get a load of this welfare loving sponge....a blight on society. http://t.co/rrZbZGO48N

---
Target: 0 (not real disaster)
Text:
#PBBan (Temporary:300) fighterdena @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason

---
Target: 1 (real disaster)
Text:
@TheSmallClark 'He'll kill me instead if he survived the shot. I don't exactly know. I fled the scene and pulled the trigger with---

---
Target: 0 (not real disaster)
Text:
One thing you can be sure of. There will never be bush fires in Scotland as the ground is always soaking wet????

---
Target: 0 (not real disaster)
Text:
@Trollkrattos Juan Carlos Salvador The Secret Tips to Get 100.000 Riot Points LoL are out now! check the Secret on on my Bio

---


In [9]:
# Let's split the data into training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                    train_df_shuffled['target'].to_numpy(),
                                                    test_size=0.1, random_state=42)

In [10]:
# check for sizes of the split
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

In [11]:
# visualise the training data
X_train[:11], y_val[0:11]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
        'Lava Dragon Breeder! I j

## Text Preprocessing

In [12]:
# check for the average number of tokens in each tweet
avg_token = round(sum([len(i.split()) for i in train_df_shuffled['text']])/len(train_df_shuffled['text']))

In [13]:
# Hyperparameters
max_vocab_length = 10000
max_length = avg_token

# create an instance of the TextVectorization Layer
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [14]:
# adapt the tokenizer to the training samples
text_vectorizer.adapt(X_train)

In [15]:
# Get an embedding layer to accurately find the relationship among the tokens
embedding = Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
)

In [16]:
# get the embedding of a random tweet
random_tweet = random.choice(X_train)
print(f'Original tweet:\n{random_tweet}\
        \n\nEmbedded version:')
embedding(text_vectorizer([random_tweet]))

Original tweet:
This the same dude that obliterated meek mill. Dont judge a book by its cover. http://t.co/BabMf0W2YW        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04010257, -0.03155156, -0.02436811, ..., -0.00300493,
          0.03704504, -0.04101551],
        [ 0.00018084,  0.02068821, -0.04576366, ...,  0.02242342,
          0.03007027, -0.00721301],
        [ 0.04515008,  0.0356558 , -0.04761125, ..., -0.00799346,
         -0.04070348,  0.01170477],
        ...,
        [ 0.01518145,  0.01624931, -0.01026131, ..., -0.03758965,
         -0.02688578, -0.02245257],
        [-0.00411649,  0.01587726, -0.01340803, ..., -0.02202151,
         -0.00292665,  0.01997269],
        [-0.03164891,  0.00409658, -0.04857785, ..., -0.03391398,
          0.02868592,  0.02371069]]], dtype=float32)>

## Helper functions

In [17]:
# a list to store all scores so that a dataframe can be created to effectively compare models against the baseline
model_score_list = []

In [18]:
# A function to take a model name, The predicted values and the actual values and return scores
def model_scores(model_name:str, y_true:list, y_preds) -> dict:
  accuracy = accuracy_score(y_true, y_preds)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_preds, average='weighted')
  results = {
      'model_name': model_name,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1': f1
  }
  model_score_list.append(results)
  return results

# A function to create a tensorboard
def create_tensorboard_callback(experinment_name, dir_name='experinments'):
  log_dir = f"{dir_name}/{experinment_name}/{tf.timestamp}"
  tensorboard_callback = TensorBoard(log_dir=log_dir)
  print(f'Saving TensorBoard log files to: {log_dir}')
  return tensorboard_callback

## Baseline Modelling
For the baseline, we will use a traditional model which in this case is the `naive bayes` classifier

In [19]:
 # create a pipeline for the model
 model_0 = Pipeline([
  ('tfidf', TfidfVectorizer()),
  ('clf', MultinomialNB())
 ])


 # fit the model
 model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [20]:
# evaluate the model
baseline_score = model_0.score(X_val, y_val)
print(f"The baseline Model acheives an accuracy of {baseline_score*100:.2f}%")

The baseline Model acheives an accuracy of 79.27%


In [21]:
# make some predictions with the baseline model
model_0_preds = model_0.predict(X_val)
model_0_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [22]:
# Get the scores for the baseline model
print(model_scores('MultinomialNB', y_val, model_0_preds))

{'model_name': 'MultinomialNB', 'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model 1:
A simple neural network with just one output layer

In [23]:
# Get the preprocessing pipeline for the model
inputs = Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = GlobalAvgPool1D()(x)
x = Dense(1, activation='sigmoid')(x)

# build the the model
model_1 = Model(inputs=inputs, outputs=x, name='model_1')

# get the summary of the model
model_1.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# compile the model
model_1.compile(loss=BinaryCrossentropy(),
                optimizer=Adam(),
                metrics=['accuracy'])

# fit the model
model_1_history = model_1.fit(X_train, y_train, batch_size=32,
                              epochs=100, validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback('Model_1')],
)

Saving TensorBoard log files to: experinments/Model_1/<function timestamp at 0x7967ed88a200>
Epoch 1/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6444 - loss: 0.6486 - val_accuracy: 0.7638 - val_loss: 0.5366
Epoch 2/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8124 - loss: 0.4604 - val_accuracy: 0.7822 - val_loss: 0.4767
Epoch 3/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8684 - loss: 0.3511 - val_accuracy: 0.7861 - val_loss: 0.4566
Epoch 4/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.8919 - loss: 0.2911 - val_accuracy: 0.7822 - val_loss: 0.4636
Epoch 5/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9130 - loss: 0.2396 - val_accuracy: 0.7795 - val_loss: 0.4783
Epoch 6/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9347 - loss: 0.1941 - val_accuracy: 0.7861 - val_loss: 0.5003
Epoch 7/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9439 - loss: 0.1694 - val_accuracy: 0.7861 - val_loss: 0.52

In [25]:
# evaluate the model
model_1.evaluate(X_val, y_val)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7085 - loss: 2.0790 


[1.848582148551941, 0.7362204790115356]

In [26]:
# make predictions with the model
model_1_preds = model_1.predict(X_val)
model_1_preds = tf.squeeze(tf.round(model_1_preds))
model_1_preds[:5]==y_val[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


<tf.Tensor: shape=(5,), dtype=bool, numpy=array([False, False,  True,  True, False])>

In [27]:
# calculate all scores of model 1 and compare to our baseline
print(model_scores('Dense', y_val, model_1_preds))

{'model_name': 'Dense', 'accuracy': 0.7362204724409449, 'precision': 0.7362825033204684, 'recall': 0.7362204724409449, 'f1': 0.7362502175903786}


In [28]:
# Create a dataframe of the score list to compare the models
pd.DataFrame(model_score_list).T

,0,1
model_name,MultinomialNB,Dense
accuracy,0.792651,0.73622
precision,0.811139,0.736283
recall,0.792651,0.73622
f1,0.786219,0.73625


## Model 2
Two LSTM layers, 1 Dense output layer

In [29]:
input = Input(shape=(1,), dtype='string')
x = text_vectorizer(input)
x = embedding(x)
x = LSTM(128)(x)
outputs = Dense(1, activation='relu', name='output_layer')(x)

In [30]:
# build the model
model_2 = Model(input, outputs, name='LSTM_model')

# compile the model
model_2.compile(loss=BinaryCrossentropy(),
                optimizer=Adam(),
                metrics=["accuracy"])

# get the summary of the model
model_2.summary()

Model: "LSTM_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,411,713 (5.39 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# fit the model
model_2_history = model_2.fit(X_train, y_train, batch_size=32,
                              epochs=100, validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback('model_2_LSTM')])

Saving TensorBoard log files to: experinments/model_2_LSTM/<function timestamp at 0x7967ed88a200>
Epoch 1/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9428 - loss: 0.2376 - val_accuracy: 0.7323 - val_loss: 2.8743
Epoch 2/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9850 - loss: 0.0299 - val_accuracy: 0.7323 - val_loss: 2.9707
Epoch 3/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9847 - loss: 0.0378 - val_accuracy: 0.7375 - val_loss: 2.7950
Epoch 4/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.9820 - loss: 0.0463 - val_accuracy: 0.7441 - val_loss: 3.0176
Epoch 5/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9855 - loss: 0.0534 - val_accuracy: 0.7297 - val_loss: 2.8768
Epoch 6/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9771 - loss: 0.0807 - val_accuracy: 0.7428 - val_loss: 3.5713
Epoch 7/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9729 - loss: 0.1910 - val_accuracy: 0.7467 - val_

In [32]:
# evaluate the model
model_2.evaluate(X_val, y_val)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7229 - loss: 3.7512


[3.26487135887146, 0.7506561875343323]

In [33]:
# make predictions with the mode and return the model scores
model_2_pred = model_2.predict(X_val)
model_2_pred = tf.squeeze(tf.round(model_2_pred))


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [34]:
# print the scores and append to the score list
print(model_scores('LSTM', y_val, model_2_pred))

{'model_name': 'LSTM', 'accuracy': 0.4763779527559055, 'precision': 0.6899604098783426, 'recall': 0.4763779527559055, 'f1': 0.5048654958276201}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
# create a dataframe to compare scores
pd.DataFrame(model_score_list).T

,0,1,2
model_name,MultinomialNB,Dense,LSTM
accuracy,0.792651,0.73622,0.476378
precision,0.811139,0.736283,0.68996
recall,0.792651,0.73622,0.476378
f1,0.786219,0.73625,0.504865


## Model 3
1. GRU cell
2. 1 dense layer with 1 unit for output

In [36]:
# get the inputs
inputs = Input(shape=(1,), dtype='string'
               )
# vectorize the inputs
x = text_vectorizer(inputs)

# embed the inputs for the model to learn a better representations
x = embedding(x)

# pass the embeddings to a GRU Layer
x = GRU(128, return_sequences=True)(x)
x = GRU(64)(x)

# pass the output to a Dense layer, whcih will be the output layer
output = Dense(1, activation = 'sigmoid', name='output_layer')(x)

# build the model
model_3 = Model(inputs, output, name='model_3')

# compile the model
model_3.compile(loss=BinaryCrossentropy(),
                optimizer=Adam(), metrics = ['accuracy'])

# get a summary of the model
model_3.summary()

Model: "model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 15, 128)             │          99,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,416,385 (5.40 MB)

 Trainable params: 1,416,385 (5.40 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# fit the model
history_3 = model_3.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val),
                        callbacks = [create_tensorboard_callback('GRU_model')])

Saving TensorBoard log files to: experinments/GRU_model/<function timestamp at 0x7967ed88a200>
Epoch 1/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 55ms/step - accuracy: 0.9356 - loss: 0.1621 - val_accuracy: 0.7493 - val_loss: 1.0428
Epoch 2/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.9841 - loss: 0.0552 - val_accuracy: 0.7428 - val_loss: 1.4053
Epoch 3/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 50ms/step - accuracy: 0.9840 - loss: 0.0370 - val_accuracy: 0.7533 - val_loss: 1.3355
Epoch 4/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.9820 - loss: 0.0424 - val_accuracy: 0.7428 - val_loss: 1.4193
Epoch 5/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.9873 - loss: 0.0248 - val_accuracy: 0.7507 - val_loss: 1.4483
Epoch 6/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.9867 - loss: 0.0280 - val_accuracy: 0.7480 - val_loss: 1.7039
Epoch 7/50
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.9810 - loss: 0.0435 - val_accuracy: 0.7467 - val_loss: 1.

In [38]:
# evaluate the model
model_3.evaluate(X_val, y_val)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7198 - loss: 2.4403


[2.1638553142547607, 0.73884516954422]

In [39]:
# make predictions with the model
model_3_pred = model_3.predict(X_val)
model_3_pred = tf.squeeze(tf.round(model_3_pred))

# get the scores for the model
print(model_scores('GRU', y_val, model_3_pred))

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
{'model_name': 'GRU', 'accuracy': 0.7388451443569554, 'precision': 0.7398979159077681, 'recall': 0.7388451443569554, 'f1': 0.7391565406820967}


In [43]:
pd.DataFrame(model_score_list).T

,0,1,2,3
model_name,MultinomialNB,Dense,LSTM,GRU
accuracy,0.792651,0.73622,0.476378,0.738845
precision,0.811139,0.736283,0.68996,0.739898
recall,0.792651,0.73622,0.476378,0.738845
f1,0.786219,0.73625,0.504865,0.739157


In [40]:
emb_layer = hub.("https://kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow1/variations/universal-sentence-encoder/versions/1", trainable=False, dtype=tf.string, name="USE")

In [42]:
# get the texts and embed them
model_3 = Sequential([
    emb_layer,
    Dense(1, activation='sigmoid')
], name='model_3')

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7967cefb0820> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)